In [124]:
#import requried modules and read in the data 
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
data=pd.read_csv("bank_marketing.csv")

In [66]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [67]:
# Seperate the target variables 
y = data['y']
X=data.drop('y',axis=1)
X.drop(['contact','poutcome'],axis=1,inplace=True)

In [68]:
# Now we have x whiccch represents our data and y which represents the target
X.head()

,age,job,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous
0,30,unemployed,married,primary,no,1787,no,no,19,oct,79,1,-1,0
1,33,services,married,secondary,no,4789,yes,yes,11,may,220,1,339,4
2,35,management,single,tertiary,no,1350,yes,no,16,apr,185,1,330,1
3,30,management,married,tertiary,no,1476,yes,yes,3,jun,199,4,-1,0
4,59,blue-collar,married,secondary,no,0,yes,no,5,may,226,1,-1,0


In [69]:
#df = df.fillna(df['Label'].value_counts().index[0])
#df["value"] = df.groupby("name").transform(lambda x: x.fillna(x.mean()))
# replace na variables in education with the most occuring variable 
data['education'] = data.groupby('job').transform(lambda x: x.fillna(x.value_counts().index[0]))
X1=data.copy()
X1['education']=X1['education'].replace('unkown','secondary')
z=X1.groupby('job')

In [70]:
X.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous'],
      dtype='object')

In [71]:
cat = ['job', 'marital', 'education', 'default','housing','loan', 'day', 'month','pdays']
a=np.linspace(0,871,10)
# replace the -1 values in pdays
X['pdays'] = np.digitize(X['pdays'],a)

In [72]:
X_cat=X[cat]
cont=['age','balance','campaign','duration']
X_cont=X[cont]
X['pdays'].unique()
print(X['pdays'].iloc[0:5])
print(pd.get_dummies(X['pdays']).iloc[0:5])

0    0
1    4
2    4
3    0
4    0
Name: pdays, dtype: int64
   0   1   2   3   4   5   6   7   8   9   10
0   1   0   0   0   0   0   0   0   0   0   0
1   0   0   0   0   1   0   0   0   0   0   0
2   0   0   0   0   1   0   0   0   0   0   0
3   1   0   0   0   0   0   0   0   0   0   0
4   1   0   0   0   0   0   0   0   0   0   0


In [73]:
X_cat = pd.get_dummies(X_cat)
X_cat.columns
X_in=pd.concat([X_cont,X_cat],axis=1)
X_in.shape
X_in.columns

Index(['age', 'balance', 'campaign', 'duration', 'day', 'pdays', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep'],
      dtype='object')

In [74]:
y.replace('no',0,inplace=True)
y.replace('yes',1,inplace=True)
X_dt = np.array(X_in)
y_dt = np.array(y)

In [110]:
#write a decision tree classifier function
dt_clf = DecisionTreeClassifier(criterion='gini',max_depth=2,min_samples_leaf=5)
#Use train_test_split tool from sklearn to split data into train and test sets
X_train,X_test,y_train,y_test = train_test_split(X_dt,y_dt,test_size=0.3,random_state=100)

In [111]:
#Now fit your classifier to your training data
dt_clf.fit(X_train,y_train)
#below output shows the hyperparameters of the classifier

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [112]:
#Now use the classifier to make a prediction on the test data
y_pred = dt_clf.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [113]:
#check the accuracy of the prediction with built in sklearn function
accuracy_score(y_pred,y_test)

0.9064112011790715

In [115]:
#The accuracy is 90%
#To improve accuracy automatically we can use gridsearch method
param_grid={'criterion':['entropy','gini'],'max_depth':[1,3,5,7],'min_samples_leaf':[1,2,3,4,5,6,7,8]}
grid_search=GridSearchCV(dt_clf,param_grid,cv=5)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'criterion': ['entropy', 'gini'], 'max_depth': [1, 3, 5, 7], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [119]:
#use the grid search best estimator to find optimized hyperparamters 
grid_search.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [ ]:
#so it seems that the accuracy does not improve from 90% regardless of the hyperparameters 

In [129]:
#Now let us run  a cross validation scheme for the original classifier and the optimized one 
#created by the gridsearch method
test = cross_val_score(dt_clf,X_dt,y_dt,cv = 10)
tests
np.mean(test)

0.891396100724765

In [130]:
test1=cross_val_score(opt_clf,X_dt,y_dt,cv=10)
test1
np.mean(test1)

0.895156674285491

In [ ]:
#We can see that the optimized classifier has a slightly better mean score
# so we will choose that one 